[View in Colaboratory](https://colab.research.google.com/github/massivetexts/MOODs/blob/master/MetaCrawl_MOODs.ipynb)

In [0]:
import pandas as pd
import time
import urllib
import json

You need a user string to download CSVs. Go to a search page on Congress.gov, click "Download", and copy the
url to the tempalte_url below.

In [0]:
# Template URL
template_url = "https://www.congress.gov/search?pageSize=25&q=%7B%22source%22%3A%22legislation%22%2C%22bill-status%22%3A%22law%22%2C%22type%22%3A%22bills%22%2C%22congress%22%3A%22110%22%7D&1ddcb92ade31c8fbd370001f9b29a7d9=628cb5675ff524f3e719b7aa2e88fe3f"
template_url = urllib.parse.urlparse(template_url)

# Change args and reencode
def construct_url(params, template_url=template_url):
    args = dict(urllib.parse.parse_qsl(template_url.query))
    args['q'] = json.dumps(params)
    parts = list(template_url)
    parts[4] = urllib.parse.urlencode(tuple(dict(args).items()))
    return urllib.parse.urlunparse(parts)

def url_to_df(url, add_cols = None):
    # Need header else rejected
    headers = {"User-Agent": 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'}
    request = urllib.request.Request(url,None,headers)
    response = urllib.request.urlopen(request)
    # Skip info in csv before the data
    df = pd.read_csv(response, skiprows=3)
    #data = response.read()
    if add_cols:
      for colname, val in add_cols:
        df[colname] = [val] * len(df)
    return df
  
opts = dict(
type = [['bills'], ['resolutions', 'joint-resolutions', 'concurrent-resolutions']],
chamber = ["House", "Senate"],
party = [['Democratic'], ['Republican', 'Independent', 'Independent Democrat']],
subject = ['Taxation', 'Government Operations and Politics', 'Health', 'Congress', 'Armed Forces and National Security', 'Foreign Trade and International Finance', 'International Affairs', 'Public Lands and Natural Resources', 'Crime and Law Enforcement', 'Transportation and Public Works', 'Social Welfare', 'Education', 'Energy', 'Agriculture and Food', 'Economics and Public Finance', 'Labor and Employment', 'Environmental Protection', 'Finance and Financial Sector', 'Commerce', 'Science, Technology, Communications', 'Immigration', 'Law', 'Housing and Community Development', 'Water Resources Development', 'Native Americans', 'Civil Rights and Liberties, Minority Issues', 'Emergency Management', 'Families', 'Animals', 'Arts, Culture, Religion', 'Sports and Recreation', 'Social Sciences and History']
)

Order:
  - type:bills, source:legislation, bill-status: passed-both
  - subject
     - (for three cases where >1000 rows) party

In [0]:
base_params = {'bill-status': 'passed-both', 'source': 'legislation', 'type': 'bills'}
# Other potential statuses: 'law' denotes just the bills that became law, 'introduced'
# gets everything. Currently using a medium-wide net with 'passed-both':
# bills that passed both chambers

all_dfs = []
for subject in opts['subject']:
  print('Subject:', subject)
  
  params = base_params.copy()
  params['subject'] = subject
  url = construct_url(params, template_url)
  df = url_to_df(url, add_cols=[('Subject', subject)])
  if len(df) < 1000:
    all_dfs.append(df)
    # Wait 10s, easier to interupt this way:
    for i in range(10):
       time.sleep(1)
  else:
    for party in opts['party']:
      print("\tParty", party)
      params['party'] = party
      url = construct_url(params, template_url)
      df = url_to_df(url, add_cols=[('Subject', subject)])
      if len(df) < 1000:
        all_dfs.append(df)
        for i in range(10):
           time.sleep(1)
      else:
        print("Still too long:", subject, party)

alldf = pd.concat(all_dfs)
alldf.sample(3)

Subject: Taxation
Subject: Government Operations and Politics
	Party ['Democratic']
	Party ['Republican', 'Independent', 'Independent Democrat']
Subject: Health
Subject: Congress
Subject: Armed Forces and National Security
Subject: Foreign Trade and International Finance
Subject: International Affairs
Subject: Public Lands and Natural Resources
	Party ['Democratic']
	Party ['Republican', 'Independent', 'Independent Democrat']
Subject: Crime and Law Enforcement
Subject: Transportation and Public Works
Subject: Social Welfare
Subject: Education
Subject: Energy
Subject: Agriculture and Food
Subject: Economics and Public Finance
Subject: Labor and Employment
Subject: Environmental Protection
Subject: Finance and Financial Sector
Subject: Commerce
Subject: Science, Technology, Communications
Subject: Immigration
Subject: Law
Subject: Housing and Community Development
Subject: Native Americans
Subject: Civil Rights and Liberties, Minority Issues
Subject: Emergency Management
Subject: Familie

,Legislation Number,URL,Congress,Amends Bill,Title,Sponsor,Date of Introduction,Date Offered,Date Submitted,Date Proposed,Number of Cosponsors,Committees,Latest Action Date,Latest Action,Subject
121,H.R. 7477,https://www.congress.gov/bill/96th-congress/ho...,96th Congress (1979-1980),NaN,A bill to amend the Internal Revenue Code of 1...,"Rep. Ullman, Al [D-OR-2]",06/03/1980,NaN,NaN,NaN,0,"House - Ways and Means, Ways and Means | Senat...",07/01/1980,Public Law 96-298.,Taxation
203,H.R. 3120,https://www.congress.gov/bill/104th-congress/h...,104th Congress (1995-1996),NaN,"To amend title 18, United States Code, with re...","Rep. Fox, Jon D. [R-PA-13]",03/20/1996,NaN,NaN,NaN,0,"House - Judiciary, Judiciary",10/01/1996,Became Public Law No: 104-214.,Crime and Law Enforcement
106,H.R. 1197,https://www.congress.gov/bill/105th-congress/h...,105th Congress (1997-1998),NaN,Plant Patent Amendment Act of 1998,"Rep. Smith, Robert [R-OR-2]",03/20/1997,NaN,NaN,NaN,2,House - Judiciary,10/27/1998,Became Public Law No: 105-289.,Commerce


**Note**: There seem to be bills without subject areas. At the time of writing, congress.gov lists 11,078 Legislation/Bill/Passed Both results, but we only scraped 9011. The counts per category (e.g. "Government Operations and Politics": 1408) are all correct, suggesting 2000 results without a subject.

In [0]:
len(alldf)

9011

In [0]:

alldf.Subject.value_counts()

Government Operations and Politics             1408
Public Lands and Natural Resources             1305
Armed Forces and National Security              678
Economics and Public Finance                    490
Transportation and Public Works                 469
Native Americans                                382
Health                                          378
International Affairs                           367
Crime and Law Enforcement                       319
Agriculture and Food                            266
Commerce                                        266
Finance and Financial Sector                    247
Energy                                          234
Water Resources Development                     206
Foreign Trade and International Finance         191
Environmental Protection                        190
Education                                       181
Law                                             180
Science, Technology, Communications             179
Taxation    

## Save Data to a Google Spreadsheet

In [0]:
!pip install --upgrade -q gspread

In [0]:
# Saving
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [0]:
#@title Save to Google Docs
# Write DataFrame to first page of sheet
name = 'MOODs Bill Meta' #@param{type:"string"}
try:
  sh = gc.open(name)
except:
  sh = gc.create(name)

In [0]:
gc.import_csv(sh.id, alldf.to_csv(None))

## Loading Data from Spreadsheet

In [0]:
records = sh.sheet1.get_all_records()
testdf = pd.DataFrame.from_records(records)
len(testdf)

9011

In [0]:
testdf.sample()['URL'].values

array(['https://www.congress.gov/bill/106th-congress/house-bill/449'],
      dtype=object)

In [0]:
#text is in <pre id="billTextContainer">...</pre>

In [0]:
from bs4 import BeautifulSoup

In [0]:
#I used the actual url, I tried testdf.sample... but it didn't work! I could extract the bill text anyway!

url = "https://www.congress.gov/bill/106th-congress/house-bill/449/text"
hdr = {'User-Agent': 'Mozilla/5.0'}
req = urllib.request.Request(url, headers=hdr)
page = urllib.request.urlopen(req)

In [0]:
soup = BeautifulSoup(page, 'html.parser')

In [0]:
#can do soup.find('pre').text as well but it gathers everything in one horizontal line!

soup.find('pre')

<pre id="billTextContainer">[106th Congress Public Law 131]
[From the U.S. Government Printing Office]


&lt;DOC&gt;
[DOCID: f:publ131.106]


[[Page 1677]]

            GATEWAY VISITOR CENTER AUTHORIZATION ACT OF 1999

[[Page 113 STAT. 1678]]

Public Law 106-131
106th Congress

                                 An Act


 
    To authorize the Gateway Visitor Center at Independence National 
 Historical Park, and for other purposes. &lt;&lt;NOTE: Dec. 7, 1999 -  [H.R. 
                                 449]&gt;&gt; 

    Be it enacted by the Senate and House of Representatives of the 
United States of America in Congress assembled, &lt;&lt;NOTE: Gateway Visitor 
Center Authorization Act of 1999.&gt;&gt; 

SECTION 1. &lt;&lt;NOTE: 16 USC 407m note.&gt;&gt;  SHORT TITLE.

    This Act may be cited as the ``Gateway Visitor Center Authorization 
Act of 1999''.

SEC. 2. &lt;&lt;NOTE: 16 USC 407m note.&gt;&gt;  FINDINGS AND PURPOSE.

    (a) Findings.--The Congress finds the following:
        